# Table of Contents
* [Exploring fenaroli parsing](#Exploring-fenaroli-parsing)
	* [Loading plaintext generated by slate (on AWS instance)](#Loading-plaintext-generated-by-slate-%28on-AWS-instance%29)
	* [RE for flavor compounds](#RE-for-flavor-compounds)
	* [RE for ingredients](#RE-for-ingredients)


In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [3]:
import cPickle as pickle
import slate 
import re

In [6]:
import matplotlib.pylab as plt
import matplotlib as mpl
plt.rcParams['figure.figsize'] = (12.0, 6.0)
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default

# Exploring fenaroli parsing

## Loading plaintext generated by slate (on AWS instance)

In [296]:
with open('../data/doc_joined.pkl', 'r') as f:
    joined = pickle.load(f)

In [295]:
cleaned_joined = joined.replace('.', ' ')

In [542]:
# cleaned_joined[:int(len(cleaned_joined)*0.2)]

In [380]:
# cleaned_joined[int(len(cleaned_joined)*0.6):int(len(cleaned_joined)*0.7)]

In [375]:
with open ('joined_inspect', 'w') as f:
    f.write(cleaned_joined)

entries I'm interested in start at split doc 17

## RE for flavor compounds and ingredients

This is the ultimate regex pattern. It finds the natural occurence, and searches below for the next flavor compound
'Natural occurrence: (.+)\n{2,5}[\x0c]?(.*[A-Z]{2,20}.*)\n+'


x \n\n335 00\n\n100 00\n\n150 00\n\n31 00\n\nSynthesis: From anisole and acetyl chloride in the presence of aluminum chloride and carbon disulfide; from anisole and acetic acid \nin the presence of boron trifluoride \n\nAroma threshold values: n/a\n\nTaste threshold values: Taste characteristics at 10 ppm: sweet, anisic, fruity, cherry with powdery vanilla nuances \n\nNatural  occurrence:  Reported  found  in  European  cranberry  (Vaccinium  oxycoccus  L ),  guava  fruit  (Psidium  guajava  L ),  Vitis \nlabrusca L , tomato, anise (Pimpinella anisum L ), mentha oils, grilled and roasted beef, sherry, cloudberry (Rubus chamaemorus \nL ), salted and pickled plums, Illicium verum and black chokeberry (Aronia melanocarpa ell ) \n\nACETIC ACID\n\nSynonyms: Acetic acid, glacial [USAN:JAN]; Ethanoic acid; Acetic acid glacial; Acetic acid (aqueous solution); Ethylic acid; \nMethanecarboxylic acid; Pyroligneous acid; Vinegar acid; Vosol; Acetic acid, of a concentration of more than 10%, by weight of \nacetic acid\n\nCAS No :\n\nCoE No :\n\n64-19-7\n\n2\n\nFL No :\n\n08 002\n\nEINECS No :\n\n200-580-7\n\nFEMA No :\n\nJECFA No :\n\n2006\n\n81\n\nNAS No :\n\nE No :\n\n2006\n\n260\n\nDescription: Acetic acid, CH3COOH, is a colorless, volatile liquid at ambient temperatures  The pure compound, glacial acetic acid, \n\nowes its name to its ice-like crystalline appearance at 15 6\xc2\


In [609]:
# sq = re.compile('Natural occurrence:\s*(.+)\n{1,5}')
sq = re.compile('Natural occurrence:(.+)\n\n[\x0c]?(.*?[A-Z]{2,20}.*?)\n\n(.*?[A-Z]{2,20}.*?)?\n?\n?')
raw_ingredients = re.findall(sq, cleaned_joined)

ingredient_series = pd.Series([i[1] for i in raw_ingredients])

In [610]:
len(raw_ingredients)

In [603]:
raw_ingredients

In [545]:
stop_strings = ingredient_series.value_counts()[:10].index[:4]

top_terms = list(ingredient_series.value_counts().index.values[:10])
stop_strings=top_terms[:4]+ top_terms[5:7] + [top_terms[8]]

In [546]:
# cleaned_ingredients = [comp.strip() for comp in raw_ingredients if comp not in stop_strings]
# cleaned_ingredients[:10]

In [639]:
with_three = [i for i in ingreient_tuple_list if i[2]]

In [640]:
len(with_three)

In [638]:
ingreient_tuple_list = []
for i in raw_ingredients:
    if i[2][:7] == 'Synonym':
        ingreient_tuple_list.append((i[0],i[1],''))
    else:
        ingreient_tuple_list.append(i)                 

## Here I have the material I need to build the ingredient:compound mapping

In [615]:
for i in raw_ingredients:
    print i[0], '\n', i[1], '\n', i[2]

In [634]:
ingreient_tuple_list

In [641]:
with_three

In [644]:
flavor_dict = defaultdict(list)
for i in ingreient_tuple_list: